# Medidas de actividad por electrodo: Distribuciones de prob de señal, desviaciones y modas.

Veamos. Franco dice que hay una distribucion trimodal en los datos. Cada moda indica la superposición de otra señal. Una es el ruido de fondo. La mas violenta debe ser la actividad real.

In [1]:
using MAT 
DatosAparato=matopen("propagacion.mat");
matread("propagacion.mat")

Dict{ASCIIString,Any} with 4 entries:
  "SamplingFrequency" => 7022.0
  "StopFrame"         => 5.549486e6
  "StartFrame"        => 5.545273e6
  "RawMatrix"         => [4125.0 2.01416015625 -28.1982421875 -10.07080078125 -…

La siguiente variable contiene TODO el arreglo de los datos, cuyas longitudes son espacio, espacio y tiempo.

In [2]:
LasMedicionesCrudas=read(DatosAparato, "RawMatrix");
size(LasMedicionesCrudas)


(64,64,4213)

La actividad mas interesante se da entre los cuadros 1300 y 2700 aprox, vamos a usar solo esos.

In [7]:
LasMedicionesCrudas=LasMedicionesCrudas[:,:,1300:2700];
size(LasMedicionesCrudas)

(64,64,1401)

Hay un par de electros saturados, que marcan voltajes absurdamente altos. Esos los vamos a matar con una funcion umbral.

In [8]:
function umbral(x)
    result=((abs(x)<1)||(abs(x)>500)?0:x)
end;

In [9]:
DatosInteresantes=map(umbral, LasMedicionesCrudas);

In [10]:
function desviaporventanas(data, param)
    ateonde=ceil(param/2)
    tantos=length(data)
    result=zeros(data)
    for j=ateonde:(tantos-ateonde)
        chusco=std(data[j-ateonde+1:j+ateonde])
        result[j]=chusco
    end
    return result 
end

function mediaporventana(data,param)
    ateonde=ceil(param/2)
    tantos=length(data)
    result=zeros(data)
    for j=ateonde:(tantos-ateonde)
        chusco=mean(data[j-ateonde+1:j+ateonde])
        result[j]=chusco
    end
    return result
end

mediaporventana (generic function with 1 method)

Varias medidas estadisticas que vamos a usar, desviacion estandar total, por canal, etc.

In [7]:
sigmaTotalAbsoluta=std(DatosInteresantes);
DesviacionPorCanal=zeros(64,64)
for j=1:64
    for k=1:64
        ChorizoExemplo=reshape(DatosInteresantes[j,k,:],1401)
        DesviacionPorCanal[j,k]=std(ChorizoExemplo)
    end
end

In [8]:
writedlm("UmpaLoompa.dat", DesviacionPorCanal-sigmaTotalAbsoluta);


In [8]:
Mascara=zeros(DesviacionPorCanal)
Mascara=map(x->((x<1)||(x>25))?0:1, DesviacionPorCanal-sigmaTotalAbsoluta);
writedlm("mascara.dat", Mascara)

Voy  a considerar ahora solo los datos que estan en la mascara. Checate como Julia hace la operacion elemento por elemento!

In [10]:
DatosFiltrados=zeros(DatosInteresantes)
for t=1:1401
    DatosFiltrados[:,:,t]=DatosInteresantes[:,:,t].*Mascara
end

In [11]:
zacatito=zeros(Float64,(64,64))
for t=1:1401
    zacatlan="DatosEnmascarados05fadeout-$t.dat"
    zacatito+=0.5+DatosFiltrados[:,:,t]
    writedlm(zacatlan,zacatito )
end

In [12]:
VarianciasTemporales=zeros(64,64);
for j=1:64
    for k=1:64
         VarianciasTemporales[j,k]=std(DatosInteresantes[j,k,:])
    end
end

In [14]:
writedlm("VarianciasTemporals01.dat", VarianciasTemporales);

In [15]:
MediasTemporales=zeros(64,64);
for j=1:64
    for k=1:64
         MediasTemporales[j,k]=mean(DatosInteresantes[j,k,:])
    end
end

In [16]:
writedlm("MediasTemporals01.dat", MediasTemporales);

In [17]:
sigmaTotalAbsoluta

16.515387990287376

In [39]:
promediototal=mean(DatosFiltrados)

-0.09987307054327693

In [19]:
VarianciasVentaneadas=zeros(DatosFiltrados)
for j=1:64
    for k=1:64
        ChorizoExemplo=reshape(DatosFiltrados[j,k,:],1401)
        VarianciasVentaneadas[j,k,:]=desviaporventanas(ChorizoExemplo,350)
    end
end

In [20]:
MediasVentaneadas=zeros(DatosFiltrados)
for j=1:64
    for k=1:64
        if (Mascara[j,k]==1)
            ChorizoExemplo=reshape(DatosFiltrados[j,k,:],1401)
            MediasVentaneadas[j,k,:]=mediaporventana(ChorizoExemplo,350)
        end
    end
end

In [21]:
zacatito=zeros(Float64,(64,64))
for t=1:1051
    zacatlan="MediasFiltradaLocal350-$t.dat"
    zacatito=VarianciasVentaneadas[:,:,t]
    writedlm(zacatlan,zacatito )
end

In [48]:
ActividadNegativa=zeros(DatosFiltrados);
ActividadPositiva=zeros(DatosFiltrados);
SpikeCountPositivo=zeros(64,64);
SpikeCountNegativo=zeros(64,64);
SpikeCountTotal=zeros(64,64;)
for j=1:64
    for k=1:64
        for t=1:1051
            if(abs(DatosFiltrados[j,k,t]-promediototal)>2*sigmaTotalAbsoluta)
                SpikeCountTotal[j,k]+=1;    
                if(MediasVentaneadas[j,k,t]<0)      
                    ActividadNegativa[j,k,t]=DatosFiltrados[j,k,t]
                    SpikeCountNegativo[j,k]+=1
                else
                    ActividadPositiva[j,k,t]=DatosFiltrados[j,k,t]
                    SpikeCountPositivo[j,k]+=1
                end
            end
            
        end
    end
end

In [49]:
writedlm("SpikeCountNegativo.dat", SpikeCountNegativo)
writedlm("SpikeCountPositivo.dat", SpikeCountPositivo)
writedlm("SpikeCountTotal.dat", SpikeCountTotal)

In [53]:
SpikeCountTotal[30,60],
Mascara[30,60]


(118.0,1)

In [67]:
testingsmooth=reshape(DatosFiltrados[30,60,:],1401);
writedlm("DatosExemplo3060.dat", testingsmooth);
suavizados=mediaporventana(testingsmooth,50);
writedlm("DatosSmoothed01.dat", suavizados);
suavizaditos=mediaporventana(testingsmooth,24);
suavizadisimos=mediaporventana(testingsmooth,350);
writedlm("DatospocoSmoothed01.dat", suavizaditos);
writedlm("DatosmuySmoothed01.dat", suavizadisimos);

In [66]:
desviacionexemplo=desviaporventanas(testingsmooth,50);
writedlm("DesviacionesExemplo01.dat", desviacionexemplo);

In [68]:
pocopocosuaves=mediaporventana(testingsmooth,10);
writedlm("MuypocoSuaves01.dat", pocopocosuaves);

In [41]:
zacatito=zeros(Float64,(64,64))
for t=1:1051
    zacatlan="PseudoSpikeCountNeg-$t.dat"
    zacatito=ActividadNegativa[:,:,t]
    writedlm(zacatlan,zacatito )
    zacatlan="PseudoSpikeCountPos-$t.dat"
    zacatito=ActividadPositiva[:,:,t]
    writedlm(zacatlan,zacatito )
end


In [24]:
Salida=open("DatosCMNegativo01.dat", "w");

In [25]:
for t=1:1051
    masa=sum(ActividadNegativa[:,:,t]);
    xmasa=0.0
    ymasa=0.0
    for j=1:64
        xmasa+=j*sum(ActividadNegativa[j,:,t])
        ymasa+=j*sum(ActividadNegativa[:,j,t])
    end
    xmasa=xmasa/masa
    ymasa=ymasa/masa
    write(Salida,join((xmasa,ymasa,masa), "\t"), "\n" )
end

In [26]:
close(Salida);
Salida=open("DatosCMPositivo01.dat", "w")

IOStream(<file DatosCMPositivo01.dat>)

In [27]:
for t=1:1051
    masa=sum(ActividadPositiva[:,:,t]);
    xmasa=0.0
    ymasa=0.0
    for j=1:64
        xmasa+=j*sum(ActividadPositiva[j,:,t])
        ymasa+=j*sum(ActividadPositiva[:,j,t])
    end
    xmasa=xmasa/masa
    ymasa=ymasa/masa
    write(Salida,join((xmasa,ymasa,masa), "\t"), "\n" )
end

In [28]:
close(Salida)

In [30]:
paescribirunfolded=sort(paescribir);
paescribirunfolded=diff(paescribirunfolded); #checa el metodo diff

LoadError: paescribir not defined
while loading In[30], in expression starting on line 1

In [31]:
writedlm("DiferenciasVariancias01.dat", paescribirunfolded)

LoadError: paescribirunfolded not defined
while loading In[31], in expression starting on line 1

In [32]:
otrasbins=range(0,0.005,200)
histodiff=hist(paescribirunfolded,otrasbins)
cruac=DataFrame();
cruac[1]=collect(otrasbins[1:end-1]);
cruac[2]=histodiff[2];
writetable("HistoDiffVariancias.dat", cruac, separator='\t')

LoadError: paescribirunfolded not defined
while loading In[32], in expression starting on line 2

In [33]:
Distributions.fit(Distributions.Gamma, paescribirunfolded)

LoadError: Distributions not defined
while loading In[33], in expression starting on line 1

In [34]:
sum(cruac[2])

LoadError: cruac not defined
while loading In[34], in expression starting on line 1

Okey, vamos a seguir Francos Way a partir de ahora. Veamos que endiabladas cosas esas funcionan mas o menos menos chungo

In [35]:
Mascara=zeros(Variancias)
Mascara=map(x->((x<26)&&(x>17))?1:0, Variancias);
writedlm("mascara.dat", Mascara)

LoadError: Variancias not defined
while loading In[35], in expression starting on line 1

In [36]:
?std

INFO: Loading help data...


Base.std(v[, region])

   Compute the sample standard deviation of a vector or array "v",
   optionally along dimensions in "region". The algorithm returns an
   estimator of the generative distribution's standard deviation under
   the assumption that each entry of "v" is an IID drawn from that
   generative distribution. This computation is equivalent to
   calculating "sqrt(sum((v - mean(v)).^2) / (length(v) - 1))".
   Note: Julia does not ignore "NaN" values in the computation. For
   applications requiring the handling of missing data, the
   "DataArray" package is recommended.


In [37]:
std(LasMedicionesCrudas)

80.68605707215762